In [2]:
#import necessary packages- pandas, numpy, & sqlalchemy
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect

In [3]:
#create an engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [15]:
#create an inspector to perform database schema inspection
inspector = inspect(engine)

In [17]:
#select and print table_names
table_names = inspector.get_table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [20]:
#Query Bookings table, print first 5 rows
df = pd.read_sql_query("SELECT * FROM Bookings", engine)
print(df.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


In [24]:
#Query Facilities table, print first 5 rows
df = pd.read_sql_query("SELECT * FROM Facilities", engine)
print(df.head())

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


In [25]:
#Query Members table, print first 5 rows
df = pd.read_sql_query("SELECT * FROM Members", engine)
print(df.head())

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


In [40]:
#QUESTIONS 
#Q1: Some of the facilities charge a fee to members, but some do not.
#Write a SQL query to produce a list of the names of the facilities that do. 

In [42]:
df = pd.read_sql_query("SELECT name FROM Facilities WHERE membercost >0", engine)
print(df)

             name
0  Tennis Court 1
1  Tennis Court 2
2  Massage Room 1
3  Massage Room 2
4    Squash Court


In [43]:
#Q2: How many facilities do not charge a fee to members? 

In [46]:
df = pd.read_sql_query("SELECT COUNT(name) AS facility_count FROM Facilities WHERE membercost = 0.0", engine)
print(df)

   facility_count
0               4


In [53]:
#Q3: Write an SQL query to show a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost.
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question.

In [54]:
df = pd.read_sql_query("SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost < 0.20*(monthlymaintenance)", engine)
print(df)

   facid             name  membercost  monthlymaintenance
0      0   Tennis Court 1         5.0                 200
1      1   Tennis Court 2         5.0                 200
2      2  Badminton Court         0.0                  50
3      3     Table Tennis         0.0                  10
4      4   Massage Room 1         9.9                3000
5      5   Massage Room 2         9.9                3000
6      6     Squash Court         3.5                  80
7      7    Snooker Table         0.0                  15
8      8       Pool Table         0.0                  15


In [55]:
#Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
#Try writing the query without using the OR operator.

In [56]:
df = pd.read_sql_query("SELECT * FROM Facilities WHERE facid IN (1,5)", engine)
print(df)

   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


In [57]:
#Q5: Produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100. Return the name and monthly maintenance of the facilities
#in question. 

In [60]:
#Note when using CASE WHEN statement, you need to place a common after the last column selected in the SELECT statement (i.e. right before CASE WHEN)
df = pd.read_sql_query("SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS facility_label FROM Facilities ORDER BY monthlymaintenance", engine)
print(df)

              name  monthlymaintenance facility_label
0     Table Tennis                  10          cheap
1    Snooker Table                  15          cheap
2       Pool Table                  15          cheap
3  Badminton Court                  50          cheap
4     Squash Court                  80          cheap
5   Tennis Court 1                 200      expensive
6   Tennis Court 2                 200      expensive
7   Massage Room 1                3000      expensive
8   Massage Room 2                3000      expensive


In [61]:
#Q6: You'd like to get the first and last name of the last member(s)
#who signed up. Try not to use the LIMIT clause for your solution. 

In [68]:
df = pd.read_sql_query("SELECT m.firstname, m.surname FROM Bookings as b INNER JOIN Members AS m ON b.memid = m.memid WHERE starttime IN (SELECT MAX(starttime) FROM BOOKINGS)", engine)
print(df)

  firstname            surname
0     GUEST              GUEST
1     Henry  Worthington-Smyth


In [69]:
#Q7: Produce a list of all members who have used a tennis court.
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name. 

In [78]:
#Note || is CONCAT () equivalent in sqlite environment
q7_query = "SELECT surname || ' ' || firstname AS member_name, name FROM Members AS m INNER JOIN Bookings AS b ON m.memid = b.memid INNER JOIN Facilities AS f ON b.facid = f.facid WHERE b.facid IN (0,1) GROUP BY member_name, name ORDER BY member_name"
df = pd.read_sql_query(q7_query, engine)
print(df)

          member_name            name
0      Bader Florence  Tennis Court 1
1      Bader Florence  Tennis Court 2
2          Baker Anne  Tennis Court 1
3          Baker Anne  Tennis Court 2
4       Baker Timothy  Tennis Court 1
5       Baker Timothy  Tennis Court 2
6          Boothe Tim  Tennis Court 1
7          Boothe Tim  Tennis Court 2
8      Butters Gerald  Tennis Court 1
9      Butters Gerald  Tennis Court 2
10        Coplin Joan  Tennis Court 1
11      Crumpet Erica  Tennis Court 1
12         Dare Nancy  Tennis Court 1
13         Dare Nancy  Tennis Court 2
14      Farrell David  Tennis Court 1
15      Farrell David  Tennis Court 2
16     Farrell Jemima  Tennis Court 1
17     Farrell Jemima  Tennis Court 2
18        GUEST GUEST  Tennis Court 1
19        GUEST GUEST  Tennis Court 2
20    Genting Matthew  Tennis Court 1
21          Hunt John  Tennis Court 1
22          Hunt John  Tennis Court 2
23        Jones David  Tennis Court 1
24        Jones David  Tennis Court 2
25      Jone

In [79]:
#Q8: Produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30. Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries. */

In [88]:
#Note when using CASE WHEN statement, you need to place a common after the last column selected in the SELECT statement (i.e. right before CASE WHEN)
q8_query = "SELECT surname || ' ' || firstname AS member_name, name, CASE WHEN m.memid=0 THEN guestcost*slots ELSE membercost*slots END AS cost FROM Members AS m INNER JOIN Bookings as b ON m.memid = b.memid INNER JOIN Facilities AS f ON b.facid = f.facid WHERE starttime LIKE '2012-09-14%' AND cost > 30 ORDER BY cost DESC"
df = pd.read_sql_query(q8_query, engine)
print(df)

       member_name            name   cost
0      GUEST GUEST  Massage Room 2  320.0
1      GUEST GUEST  Massage Room 1  160.0
2      GUEST GUEST  Massage Room 1  160.0
3      GUEST GUEST  Massage Room 1  160.0
4      GUEST GUEST  Tennis Court 2  150.0
5      GUEST GUEST  Tennis Court 1   75.0
6      GUEST GUEST  Tennis Court 1   75.0
7      GUEST GUEST  Tennis Court 2   75.0
8      GUEST GUEST    Squash Court   70.0
9   Farrell Jemima  Massage Room 1   39.6
10     GUEST GUEST    Squash Court   35.0
11     GUEST GUEST    Squash Court   35.0


In [94]:
#Q9: This time, produce the same result as in Q8, but using a subquery.

In [95]:
q9_query = "SELECT facility, member, cost FROM (SELECT m.surname || ' ' || m.firstname AS member, f.name AS facility, CASE WHEN m.memid = 0 THEN guestcost*slots ELSE membercost*slots END AS cost FROM Members AS m INNER JOIN Bookings AS b ON m.memid = b.memid INNER JOIN Facilities AS f ON b.facid = f.facid WHERE starttime LIKE '2012-09-14%') AS booking_costs WHERE cost > 30 ORDER BY cost DESC"
df = pd.read_sql_query(q9_query, engine)
print(df)

          facility          member   cost
0   Massage Room 2     GUEST GUEST  320.0
1   Massage Room 1     GUEST GUEST  160.0
2   Massage Room 1     GUEST GUEST  160.0
3   Massage Room 1     GUEST GUEST  160.0
4   Tennis Court 2     GUEST GUEST  150.0
5   Tennis Court 1     GUEST GUEST   75.0
6   Tennis Court 1     GUEST GUEST   75.0
7   Tennis Court 2     GUEST GUEST   75.0
8     Squash Court     GUEST GUEST   70.0
9   Massage Room 1  Farrell Jemima   39.6
10    Squash Court     GUEST GUEST   35.0
11    Squash Court     GUEST GUEST   35.0


In [ ]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! 

In [97]:
q10_query = "SELECT facility, SUM(revenue) AS total_revenue FROM (SELECT f.name AS facility, CASE WHEN m.memid = 0 THEN guestcost*slots ELSE membercost*slots END AS revenue FROM Members AS m INNER JOIN Bookings AS b ON m.memid = b.memid INNER JOIN Facilities as f ON b.facid = f.facid) AS facility_revenues GROUP BY facility ORDER BY total_revenue DESC"
df = pd.read_sql_query(q10_query, engine)
print(df)

          facility  total_revenue
0   Massage Room 1        50351.6
1   Massage Room 2        14454.6
2   Tennis Court 2        14310.0
3   Tennis Court 1        13860.0
4     Squash Court        13468.0
5  Badminton Court         1906.5
6       Pool Table          270.0
7    Snooker Table          240.0
8     Table Tennis          180.0


In [98]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [102]:
q11_query = "SELECT DISTINCT m.surname || ' ' || m.firstname AS member, (SELECT r.surname || ' ' || r.firstname FROM Members AS r WHERE r.memid = m.recommendedby) AS recommender FROM Members AS m ORDER BY member"
df = pd.read_sql_query(q11_query, engine)
print(df)

                     member        recommender
0            Bader Florence    Stibbons Ponder
1                Baker Anne    Stibbons Ponder
2             Baker Timothy     Farrell Jemima
3                Boothe Tim         Rownam Tim
4            Butters Gerald       Smith Darren
5               Coplin Joan      Baker Timothy
6             Crumpet Erica        Smith Tracy
7                Dare Nancy    Joplette Janice
8             Farrell David               None
9            Farrell Jemima               None
10              GUEST GUEST               None
11          Genting Matthew     Butters Gerald
12                Hunt John  Purview Millicent
13              Jones David    Joplette Janice
14            Jones Douglas        Jones David
15          Joplette Janice       Smith Darren
16           Mackenzie Anna       Smith Darren
17             Owen Charles       Smith Darren
18             Pinker David     Farrell Jemima
19        Purview Millicent        Smith Tracy
20           

In [ ]:
#Q12: Find the facilities with their usage by member, but not guests.

In [4]:
q12_query = "SELECT f.name AS facility, m.surname || ' ' || m.firstname AS member_name, SUM(slots) AS num_slots FROM Bookings AS b JOIN Facilities AS f ON b.facid = f.facid JOIN Members AS m ON b.memid = m.memid WHERE b.memid !=0 GROUP BY facility, member_name"
df = pd.read_sql_query(q12_query, engine)
print(df)

            facility      member_name  num_slots
0    Badminton Court   Bader Florence         27
1    Badminton Court       Baker Anne         30
2    Badminton Court    Baker Timothy         21
3    Badminton Court       Boothe Tim         36
4    Badminton Court   Butters Gerald         63
..               ...              ...        ...
197   Tennis Court 2     Smith Darren         57
198   Tennis Court 2       Smith Jack          3
199   Tennis Court 2      Smith Tracy          6
200   Tennis Court 2  Stibbons Ponder         96
201   Tennis Court 2     Tracy Burton          9

[202 rows x 3 columns]


In [ ]:
#Q13: Find the facilities usage by month, but not guests

In [9]:
q13_query = "SELECT f.name AS facility, STRFTIME('%m', b.starttime) AS month, SUM(slots) AS num_slots FROM Bookings AS b JOIN Facilities AS f ON b.facid = f.facid JOIN Members AS m ON b.memid = m.memid WHERE b.memid = 0 GROUP BY facility, month"
df = pd.read_sql_query(q12_query, engine)
print(df)

           facility month  num_slots
0   Badminton Court    07         15
1   Badminton Court    08         45
2   Badminton Court    09         63
3    Massage Room 1    07         98
4    Massage Room 1    08        176
5    Massage Room 1    09        246
6    Massage Room 2    07         16
7    Massage Room 2    08         64
8    Massage Room 2    09         94
9        Pool Table    07          7
10       Pool Table    08         19
11       Pool Table    09         28
12    Snooker Table    07         16
13    Snooker Table    08         10
14    Snooker Table    09         22
15     Squash Court    07        114
16     Squash Court    08        216
17     Squash Court    09        356
18     Table Tennis    07          6
19     Table Tennis    08          8
20     Table Tennis    09         22
21   Tennis Court 1    07         69
22   Tennis Court 1    08        120
23   Tennis Court 1    09        174
24   Tennis Court 2    07         84
25   Tennis Court 2    08        138
2